In [22]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel

In [23]:
load_dotenv(override=True)

True

### MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

In [3]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params) as server:
    fetch_tools = await server.list_tools()

for tool in fetch_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")

fetch: Fetches a URL from the internet and optionally extracts its contents as markdown.  Although originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.


In [4]:
puppeteer_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-puppeteer"]}

async with MCPServerStdio(params=puppeteer_params) as server:
    puppeteer_tools = await server.list_tools()

for tool in puppeteer_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")


puppeteer_navigate: Navigate to a URL
puppeteer_screenshot: Take a screenshot of the current page or a specific element
puppeteer_click: Click an element on the page
puppeteer_fill: Fill out an input field
puppeteer_select: Select an element on the page with Select tag
puppeteer_hover: Hover an element on the page
puppeteer_evaluate: Execute JavaScript in the browser console


In [5]:
puppeteer_tools[0]

Tool(name='puppeteer_navigate', description='Navigate to a URL', inputSchema={'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'URL to navigate to'}, 'launchOptions': {'type': 'object', 'description': "PuppeteerJS LaunchOptions. Default null. If changed and not null, browser restarts. Example: { headless: true, args: ['--no-sandbox'] }"}, 'allowDangerous': {'type': 'boolean', 'description': 'Allow dangerous LaunchOptions that reduce security. When false, dangerous args like --no-sandbox will throw errors. Default false.'}}, 'required': ['url']}, annotations=None)

In [6]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params) as server:
    file_tools = await server.list_tools()

for tool in file_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")


read_file: Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.
read_multiple_files: Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.
write_file: Create a new file or completely overwrite an existing file with new content. Use with caution as it will overwrite existing files without warning. Handles text content with proper encoding. Only works within allowed directories.
edit_file: Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showin

In [7]:
playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params) as server:
    playwright_tools = await server.list_tools()

for tool in playwright_tools:
    print(f"{tool.name.replace('\n', ' ')}: {tool.description.replace('\n', ' ')}")


browser_close: Close the page
browser_resize: Resize the browser window
browser_console_messages: Returns all console messages
browser_handle_dialog: Handle a dialog
browser_file_upload: Upload one or multiple files
browser_install: Install the browser specified in the config. Call this if you get an error about the browser not being installed.
browser_press_key: Press a key on the keyboard
browser_navigate: Navigate to a URL
browser_navigate_back: Go back to the previous page
browser_navigate_forward: Go forward to the next page
browser_network_requests: Returns all network requests since loading the page
browser_pdf_save: Save page as PDF
browser_take_screenshot: Take a screenshot of the current page. You can't perform actions based on the screenshot, use browser_snapshot for actions.
browser_snapshot: Capture accessibility snapshot of the current page, this is better than screenshot
browser_click: Perform click on a web page
browser_drag: Perform drag and drop between two elements
b

In [24]:
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Using Gemini API
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_NAME='gemini-2.5-flash-preview-04-17'
gemini_client = AsyncOpenAI(base_url=BASE_URL, api_key=os.getenv("GOOGLE_API_KEY"))
custom_model = OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=gemini_client)

In [ ]:
# Using Groq API
BASE_URL = "https://api.groq.com/openai/v1"
MODEL_NAME='llama-3.1-8b-instant'
groq_client = AsyncOpenAI(base_url=BASE_URL, api_key=os.getenv("GROQ_API_KEY"))
custom_model = OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=groq_client)

In [32]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, cache_tools_list=True) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, cache_tools_list=True) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model=custom_model,
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie in detail with steps, then summarize and save it in markdown to sandbox/banoffee.md")
            print(result.final_output)


I have found a great Banoffee Pie recipe, summarized it, and saved the summary in markdown format to `sandbox/banoffee.md`.

Here is the summary:

**Banoffee Pie Recipe Summary**

This no-bake Banoffee Pie features a simple graham cracker crust, a layer of dulce de leche, sliced bananas, and a generous topping of homemade whipped cream. The recipe includes instructions for making dulce de leche from sweetened condensed milk in the oven, which is considered safer than boiling the can. It's a quick and easy dessert that can be made ahead of time, though the bananas are best added closer to serving to prevent browning.

**Ingredients:**

*   **Crust:**
    *   1 and 1/2 cups (180g) graham cracker crumbs
    *   1/4 cup (50g) granulated sugar
    *   6 Tablespoons (85g) unsalted butter, melted
*   **Filling:**
    *   1 and 1/4 cups dulce de leche
    *   2 large bananas, sliced
*   **Whipped Cream:**
    *   2 cups (480ml) cold heavy cream or heavy whipping cream
    *   1/4 cup confectio